# Lab2 - Expansão de Consultas
### Hadrizia Santos
Nesta atividade será exercitada a noção de expansão de consultas. Considerando a coleção de notícias do lab passado, deve-se executar os seguintes passos:

1. Escrever uma função que receba uma coleção de documentos e retorne uma matrix de termos-termos contendo as frequências de co-ocorrência de duas palavras consecutivas no texto (bigramas).
2. Escrever uma função que receba um certo termo de consulta e a matriz construída no passo 1 acima e retorneas top-3 palavras em ordem decrescente de frequencia.
3. Expandir a consulta original com os termos retornados no passo 2 acima.
4. Fazer uma busca disjuntiva (OR) considerando a nova consulta.

E responder às perguntas:

* Quais os termos retornados para a expansão de cada consulta?
* Você acha que esses termos são de fato relacionados com a consulta original? Justifique.
* Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?
* A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?

## Importar bibliotecas e os dados
O primeiro passo é importar as bibliotecas e os dados necessários.

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
import nltk
from nltk import bigrams   
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import scipy.sparse as sps
import math
import collections
from collections import OrderedDict

#nltk.download("stopwords") 

FILE_PATH = '../data/estadao_noticias_eleicao.csv'

df = pd.read_csv(FILE_PATH, encoding = 'utf-8')
df = df.replace(np.NAN, "")

# criação de uma nova coluna para a junção do título da notícia com seu conteúdo
df['noticia'] = df.titulo + ' ' + df.subTitulo + ' ' + df.conteudo

dictionary = collections.defaultdict(list)
OR = 'or'

## 1. Construir matrix de ocorrência
**Obs:** O código abaixo que constrói a matriz de ocorrência foi copiado do repositório que está disponível em: https://github.com/allansales/information-retrieval/blob/master/Lab%202/coocurrence_matrix.ipynb

In [2]:
# O código abaixo está disponivel em: 
# https://github.com/allansales/information-retrieval/blob/master/Lab%202/coocurrence_matrix.ipynb

def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

def  generate_tokens_dataframe(noticia):
    # Removing punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    tokens_lists = noticia.apply(lambda text: tokenizer.tokenize(text.lower()))
    
    # Removing stopwords
    stopword_ = stopwords.words('portuguese')
    filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])
    
    tokens = [token for tokens_list in filtered_tokens for token in tokens_list]
    return tokens

def  generate_tokens_text(text):
    # Removing punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    tokens_lists = tokenizer.tokenize(text.lower())
    
    # Removing stopwords
    stopword_ = stopwords.words('portuguese')
    filtered_tokens = [token for token in tokens_lists if token not in stopword_]

    return filtered_tokens

def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

In [3]:
tokens = generate_tokens_dataframe(df.noticia)
matrix, vocab = co_occurrence_matrix(tokens)
consultable_matrix = matrix.tocsr()
inverted_vocab = {vocab[key]: key for key in vocab}

## 2. Retornar as top-3 palavras mais frequentes de acordo com a matriz de ocorrência
Escrever uma função que receba uma palavra do dicionário e retorne quais as 3 palavras que ocorrem mais frequentemente com o input.

In [4]:
def get_co_ocurrences(co_matrix, inv_vocab, term, N=3):
    # sparse to dense representation
    np_array = np.reshape(co_matrix[term].toarray(), -1)
    # get indice of N occurences
    return list(OrderedDict({inv_vocab[idx]: np_array[idx] for idx in (-np_array).argsort()[:N]}).keys())

## 3. Expandir a consulta original com os termos retornados no passo 2 acima.
Escrever uma função que receba a consulta, pega as 3 palavras que aparecem com maior frequência junto com esse termo e retorne uma consulta expandida.

In [5]:
def expand_query(terms):
    query = []
    terms = terms.split(' ')
    if len(terms) > 1:
            for term in terms:
                query.append(term)
                query.extend(get_co_ocurrences(consultable_matrix, inverted_vocab, vocab[term], 3))
    else:
        query.append(terms[0])
        query.extend(get_co_ocurrences(consultable_matrix, inverted_vocab, vocab[terms[0]], 3))
        
    # removing duplicates terms
    return " ".join(str(x) for x in set(query))

## 4. Fazer uma busca disjuntiva (OR) considerando a nova consulta.
Escrever uma função que recebe uma consulta de N termos e retorna uma consula disjuntiva entre os termos. O modelo utilizado será a busca binária, que foi implementado em atividades anteriores.

In [6]:
def create_indexes(tokens, docId):
    for word in tokens:
        if docId not in dictionary[word]:
            dictionary[word].append(docId)

In [7]:
# Creating inverted index
for index, row in df.iterrows():  
    tokens = generate_tokens_text(row.noticia)
    create_indexes(tokens, row.idNoticia)

In [8]:
def or_search(terms):
    result = set(dictionary[terms[0]])
    for term in terms[1:]:
        result = result.union(set(dictionary[term]))
    return list(result)

def search(search):
    search = search.lower().split(' ')
    if len(search) >= 1:
        terms_list = []
        for element in search:
            if element != OR:
                terms_list.append(element)
        return or_search(terms_list)
    else:
        raise ValueError('search must have at least a word.')

## Respondendo às perguntas
### Quais os termos retornados para a expansão de cada consulta?
As consultas a serem testadas são as seguintes: 'segundo turno', 'lava jato', 'projeto de lei', 'compra de voto' e 'empresa petrobrás'

In [26]:
queries = ['segundo turno', 'lava jato', 'projeto de lei', 'compra de voto', 'ministério público']
exp_queries = []
for query in queries:
    ext_query = expand_query(process_query(query))
    exp_queries.append(ext_query)
    print('Query original: %s' % query)
    print('Query expandida: %s' % ext_query)

Query original: segundo turno
Query expandida: é turno segundo eleição mandato lugar eleições
Query original: lava jato
Query expandida: é lava deflagrada lato jato porque polícia
Query original: projeto de lei
Query expandida: ficha político poder lei anistia projeto responsabilidade
Query original: compra de voto
Query expandida: pasadena dilma refinaria voto votos distrital presidente compra
Query original: ministério público
Query expandida: estadual é ministério saúde fazenda público federal


### Você acha que esses termos são de fato relacionados com a consulta original? Justifique.
Acredito que os termos estejam relacionados com a consulta original nas duas últimas consultas, pois quando se fala em segundo turno, é normalmente sobre eleições e todos os termos estão relacionados ao contexto. O mesmo ocorre para petrobrás, onde os demais termos se relacionam com a consulta. Já a primeira consulta expandida me surpreendeu um pouco, pois não esperava que petrobrás aparecesse tanto junto de corrupção. Outra coisa que observei é que o termo '**é**', que deveria ser considerada stopword, pois apareceu em todas as consultas.


### Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?
A consulta utilizada será uma busca binária, implementada no Lab1 desta disciplina.

In [17]:
# process query
def process_query(query):
    return ' '.join(generate_tokens_text(query))

In [34]:
result = []
exp_result = []
for query in queries:
    result.append(search(process_query(query)))
for query in exp_queries:
    exp_result.append(search(process_query(query)))
    
for i in range(len(result)):
    print('Para a consulta "%s", a busca retornou %d documentos.' % (queries[i], len(result[i])))
    print('Para a consulta expandida "%s", a busca retornou %d documentos.' % (exp_queries[i], len(exp_result[i])))

Para a consulta "segundo turno", a busca retornou 4448 documentos.
Para a consulta expandida "é turno segundo eleição mandato lugar eleições", a busca retornou 7368 documentos.
Para a consulta "lava jato", a busca retornou 530 documentos.
Para a consulta expandida "é lava deflagrada lato jato porque polícia", a busca retornou 7023 documentos.
Para a consulta "projeto de lei", a busca retornou 1971 documentos.
Para a consulta expandida "ficha político poder lei anistia projeto responsabilidade", a busca retornou 4965 documentos.
Para a consulta "compra de voto", a busca retornou 1659 documentos.
Para a consulta expandida "pasadena dilma refinaria voto votos distrital presidente compra", a busca retornou 6376 documentos.
Para a consulta "ministério público", a busca retornou 2474 documentos.
Para a consulta expandida "estadual é ministério saúde fazenda público federal", a busca retornou 7320 documentos.


Observando os resultados acima, é possível notar que o número de documentos retornados aumentou muito. Acredito que, uma vez que os termos da consulta expandida tenham sim a ver com o termo original, essa abordagem pode ser mais eficiente para recuperar os documentos que mais se relacionam com o que o usuário buscou.

### A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?

Como a expansão de consultas gera muito mais documentos retornados, ela é adequada para melhorar o recall, uma vez que o recall é o número de documentos corretos dividido pelo número de documentos que deveriam ter sido retornados. Então quando maior for o número de documentos retornados, maior o recall será.